In [ ]:
# 01 시퀀스-투-시퀀스(Sequence-to-Sequence, seq2seq)

In [3]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

lines = pd.read_csv("C:/Users/zrowx/Downloads/fra-eng/fra.txt", names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

# 60,000개의 샘플만 가지고 기계 번역기 구축

전체 샘플의 개수 : 227815


,src,tar
30891,Who broke the cup?,Qui a cassé la tasse ?
3363,I love soup.,J'adore la soupe.
34632,I've caught a cold.,J'ai contracté un rhume.
59181,Rinse with warm water.,Rincer avec de l'eau tiède.
11075,How's your job?,Comment va ton boulot ?
7870,I have to win.,Il faut que je gagne.
10300,"A beer, please.","Une bière, s'il vous plaît !"
14767,Find me a glass.,Trouvez-moi un verre.
39900,He parties too much.,Il fait trop la bamboula.
48063,I did that on Monday.,J'ai fait ça lundi.


In [4]:
# 시작을 의미하는 심볼 <sos> -> \t
# 종료를 의미하는 심볼 <eos>을 넣어주어야 함 -> \n
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

# 시작 심볼과 종료 심볼이 추가

,src,tar
48336,I hard-boiled an egg.,\t J'ai cuit un œuf dur. \n
29333,They had a choice.,\t Elles avaient le choix. \n
28977,That sounds scary.,\t Cela semble effrayant. \n
12624,She has a book.,\t Elle a un livre. \n
21081,I understand Tom.,\t Je comprends Tom. \n
59257,She has gone shopping.,\t Elle est allée faire les courses. \n
1155,I saw him.,\t Je le vis. \n
38335,You owe me a favor.,\t Tu me dois une faveur. \n
8157,I was selfish.,\t J'étais égoïste. \n
37832,When will it begin?,\t Quand est-ce que ça commence ? \n


In [5]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [6]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

# 영어와 프랑스어는 각각 80개와 104개의 문자가 존재

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [7]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

# 영어와 프랑스어는 각각 80개와 104개의 문자가 존재

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [8]:
# 각 문자에 인덱스를 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'v': 1, 'a': 2, 'C': 3, 'i': 4, '7': 5, 'z': 6, '€': 7, 'c': 8, 'T': 9, ',': 10, 'G': 11, 'N': 12, '&': 13, 'Q': 14, 't': 15, 'R': 16, 'd': 17, '%': 18, '1': 19, '8': 20, 'L': 21, 'Y': 22, 'y': 23, '!': 24, 'M': 25, '/': 26, ' ': 27, 'j': 28, 'Z': 29, '"': 30, 'X': 31, 'e': 32, 'J': 33, 'é': 34, '-': 35, 'V': 36, 'k': 37, 'E': 38, ':': 39, 'u': 40, 'B': 41, 'p': 42, 'O': 43, '?': 44, '2': 45, 'g': 46, 'b': 47, 'q': 48, 'ï': 49, 'l': 50, '3': 51, 'r': 52, 'S': 53, '4': 54, 'o': 55, '$': 56, 'A': 57, 'n': 58, 'K': 59, 'm': 60, 'x': 61, 'w': 62, 'D': 63, "'": 64, 'I': 65, '6': 66, '.': 67, '9': 68, 'H': 69, '5': 70, 'f': 71, 'h': 72, '0': 73, 's': 74, 'F': 75, '’': 76, 'U': 77, 'W': 78, 'P': 79}
{'«': 1, 'v': 2, 'û': 3, 'a': 4, 'C': 5, 'i': 6, 'ê': 7, '7': 8, 'z': 9, 'c': 10, 'T': 11, ',': 12, 'G': 13, 'N': 14, 'Q': 15, '&': 16, 't': 17, 'R': 18, 'd': 19, 'î': 20, '%': 21, '1': 22, '‽': 23, '8': 24, 'L': 25, '\u202f': 26, 'ç': 27, 'Y': 28, 'y': 29, '»': 30, '!': 31, 'M': 32, 'è': 33, 'Ô'

In [9]:
# 인덱스가 부여된 문자 집합으로부터 갖고있는 훈련 데이터에 정수 인코딩 수행
encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        # 각 char을 정수로 변환
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[11, 55, 67], [11, 55, 67], [11, 55, 67], [11, 55, 67], [69, 4, 67]]


In [10]:
# 디코더의 입력이 될 프랑스어 데이터에 대해서 정수 인코딩 수행
decoder_input = []
for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[41, 35, 45, 4, 35, 31, 35, 99], [41, 35, 32, 4, 67, 10, 92, 40, 86, 35, 99], [41, 35, 47, 76, 35, 67, 72, 50, 17, 40, 35, 31, 35, 99], [41, 35, 53, 72, 50, 60, 40, 35, 31, 35, 99], [41, 35, 70, 4, 66, 50, 17, 35, 31, 35, 99]]


In [11]:
# 디코더의 예측값과 비교하기 위한 실제값 필요
# 실제값에는 시작 심볼에 해당되는 <sos>가 있을 필요 없음
# 정수 인코딩 과정에서 <sos>를 제거
# -> 프랑스어 문장의 맨 앞에 붙어있는 '\t'를 제거
decoder_target = []
for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])
# decoder_input에서는 모든 문장의 앞에 붙어있던 숫자 1이 decoder_target에서는 제거됨

target 문장 레이블의 정수 인코딩 : [[35, 45, 4, 35, 31, 35, 99], [35, 32, 4, 67, 10, 92, 40, 86, 35, 99], [35, 47, 76, 35, 67, 72, 50, 17, 40, 35, 31, 35, 99], [35, 53, 72, 50, 60, 40, 35, 31, 35, 99], [35, 70, 4, 66, 50, 17, 35, 31, 35, 99]]


In [12]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [13]:
# 영어 데이터의 샘플은 전부 길이가 22이 되도록 패딩
# 프랑스어 데이터의 샘플은 전부 길이가 76이 되도록 패딩
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [14]:
# 모든 값에 대해서 원-핫 인코딩을 수행
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [15]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import numpy as np

encoder_inputs = Input(shape=(None, src_vocab_size))
# return_state=True로 설정 -> 인코더에 입력을 넣으면 내부 상태를 리턴
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [16]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [17]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 185s 244ms/step - loss: 0.7267 - val_loss: 0.6295
Epoch 2/40
750/750 [==============================] - 201s 268ms/step - loss: 0.4459 - val_loss: 0.5088
Epoch 3/40
750/750 [==============================] - 185s 247ms/step - loss: 0.3720 - val_loss: 0.4496
Epoch 4/40
750/750 [==============================] - 179s 239ms/step - loss: 0.3308 - val_loss: 0.4136
Epoch 5/40
750/750 [==============================] - 178s 238ms/step - loss: 0.3029 - val_loss: 0.3903
Epoch 6/40
750/750 [==============================] - 184s 245ms/step - loss: 0.2824 - val_loss: 0.3745
Epoch 7/40
750/750 [==============================] - 177s 237ms/step - loss: 0.2663 - val_loss: 0.3594
Epoch 8/40
750/750 [==============================] - 175s 233ms/step - loss: 0.2532 - val_loss: 0.3523
Epoch 9/40
750/750 [==============================] - 177s 236ms/step - loss: 0.2423 - val_loss: 0.3462
Epoch 10/40
750/750 [==============================] - 177s 236m

In [18]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [19]:
# 인코더 정의

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [20]:
# 인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar를 만들기
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [21]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [22]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Cassez-vous. 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut. 
1/1 [==============================] - 0s 17ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Compres ! 
1/1 [==============================] - 0s 15ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Rentrez chez vous. 
1/1 [==============================] - 0s 18ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: En route. 


In [23]:
# 02 Word-Level 번역기 만들기(Neural Machine Translation (seq2seq) Tutorial)

In [26]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [27]:
num_samples = 33000

In [28]:
# 전처리 함수
def unicode_to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백 추가.
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [29]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [40]:
# 교사 강요(Teacher Forcing)
# 레이블에 해당되는 출력 시퀀스를 따로 분리하여 저장
# 입력 시퀀스에는 시작을 의미하는 토큰인 <sos>를 추가
# 출력 시퀀스에는 종료를 의미하는 토큰인 <eos>를 추가
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("C:/Users/zrowx/Downloads/fra-eng/fra.txt", "r", encoding='UTF8') as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [41]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


In [42]:
# 케라스 토크나이저를 통해 단어 집합을 생성
# 정수 인코딩을 진행
# 패딩을 진행
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [43]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

# 영어 문장의 길이는 8-----------------------------------------------------------------------------------------------------------
# 프랑스어 문장의 길이는 16

인코더의 입력의 크기(shape) : (33000, 7)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [44]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4481, 프랑스어 단어 집합의 크기 : 7873


In [45]:
# 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어줌
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [46]:
# 순서가 섞인 정수 시퀀스 리스트
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [30307 25711 31026 ... 25249 25944 32152]


In [47]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [48]:
print(encoder_input[30997])
print(decoder_input[30997])
print(decoder_target[30997])

# decoder_target은 데이터의 구조상으로 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>을 제외하면 동일한 정수 시퀀스를 가져야

[  2 341   3   9 253   1   0]
[  2   4   9 100  32 265   1   0   0   0   0   0   0   0   0   0]
[  4   9 100  32 265   1   3   0   0   0   0   0   0   0   0   0]


In [49]:
# 훈련 데이터의 10%를 테스트 데이터로 분리
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [50]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [51]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 7)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 7)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


In [52]:
from keras.layers import Input, LSTM, Embedding, Dense, Masking
from keras.models import Model

In [53]:
# 임베딩 벡터의 차원과 LSTM의 은닉 상태의 크기를 64로 사용
embedding_dim = 64
hidden_units = 64

In [54]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [55]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [56]:
# 128개의 배치 크기로 총 50 에포크 학습
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
233/233 [==============================] - 59s 235ms/step - loss: 3.2922 - acc: 0.6187 - val_loss: 2.0235 - val_acc: 0.6192
Epoch 2/50
233/233 [==============================] - 63s 270ms/step - loss: 1.8461 - acc: 0.6749 - val_loss: 1.7491 - val_acc: 0.7362
Epoch 3/50
233/233 [==============================] - 61s 262ms/step - loss: 1.6531 - acc: 0.7432 - val_loss: 1.6096 - val_acc: 0.7443
Epoch 4/50
233/233 [==============================] - 61s 261ms/step - loss: 1.5239 - acc: 0.7566 - val_loss: 1.4936 - val_acc: 0.7589
Epoch 5/50
233/233 [==============================] - 61s 264ms/step - loss: 1.4133 - acc: 0.7682 - val_loss: 1.3994 - val_acc: 0.7729
Epoch 6/50
233/233 [==============================] - 61s 260ms/step - loss: 1.3168 - acc: 0.7860 - val_loss: 1.3183 - val_acc: 0.7911
Epoch 7/50
233/233 [==============================] - 61s 263ms/step - loss: 1.2400 - acc: 0.7988 - val_loss: 1.2606 - val_acc: 0.8017
Epoch 8/50
233/233 [==============================] - 5

In [57]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [69]:
# 결과 확인용 함수
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
            len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [70]:
# seq_to_src : 영어 문장에 해당하는 정수 시퀀스를 입력받음
# 정수로부터 영어 단어를 리턴하는 index_to_src를 통해 영어 문장으로 변환
# seq_to_tar : 프랑스어에 해당하는 정수 시퀀스를 입력받음
# 정수로부터 프랑스어 단어를 리턴하는 index_to_tar을 통해 프랑스어 문장으로 변환

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

In [71]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

1/1 [==============================] - 0s 15ms/step
입력문장 : i ve been at home . 
정답문장 : j etais a la maison . 
번역문장 : j etais a la maison . 
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
입력문장 : you re talented . 
정답문장 : vous etes talentueuse . 
번역문장 : tu es talentueuse . 
--------------------------------------------------
1/1 [==============================] - 0s 15ms/step
입력문장 : we re not welcome . 
정답문장 : nous ne sommes pas les bienvenus . 
번역문장 : nous ne sommes pas en train de la . 
--------------------------------------------------
1/1 [==============================] - 0s 16ms/step
입력문장 : tom s wounded . 
정답문장 : tom est blesse . 
번역문장 : tom est blesse . 
--------------------------------------------------
1/1 [==============================] - 0s 17ms/step
입력문장 : i am a cook . 
정답문장 : je suis cuisinier . 
번역문장 : je suis cuisinier . 
--------------------------------------------------


In [72]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
    print("번역문장 :",decoded_sentence[1:-5])
    print("-"*50)

1/1 [==============================] - 0s 15ms/step
입력문장 : what happens next ? 
정답문장 : et apres ? 
번역문장 : qu est ce que il se fait ? 
--------------------------------------------------
1/1 [==============================] - 0s 14ms/step
입력문장 : don t be so glum . 
정답문장 : ne sois pas si morose . 
번역문장 : ne le fais pas confiance en qui tom . 
--------------------------------------------------
1/1 [==============================] - 0s 16ms/step
입력문장 : tom paid cash . 
정답문장 : tom a paye en liquide . 
번역문장 : tom a paye comptant . 
--------------------------------------------------
1/1 [==============================] - 0s 14ms/step
입력문장 : tom arrives today . 
정답문장 : tom arrive aujourd hui . 
번역문장 : tom fait parti aujourd hui . 
--------------------------------------------------
1/1 [==============================] - 0s 13ms/step
입력문장 : relax a moment . 
정답문장 : detends toi un instant ! 
번역문장 : detendez vous un moment ! 
--------------------------------------------------


In [73]:
# 03 BLEU Score(Bilingual Evaluation Understudy Score)

In [74]:
import numpy as np
from collections import Counter
from nltk import ngrams

In [75]:
# 보정된 유니그램 정밀도 (Modified Unigram Precision) 구현

In [76]:
# 토큰화 된 문장(tokens)에서 n-gram을 카운트
def simple_count(tokens, n):
    return Counter(ngrams(tokens, n))

In [77]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [78]:
candidate = 'the the the the the the the'
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 7})


In [79]:
def count_clip(candidate, reference_list, n):
    # Ca 문장에서 n-gram 카운트
    ca_cnt = simple_count(candidate, n)
    max_ref_cnt_dict = dict()

    for ref in reference_list: 
        # Ref 문장에서 n-gram 카운트
        ref_cnt = simple_count(ref, n)

    # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
    for n_gram in ref_cnt: 
        if n_gram in max_ref_cnt_dict:
            max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
        else:
            max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

    return {
        # count_clip = min(count, max_ref_count)
        n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
     }

In [80]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(),list(map(lambda ref: ref.split(), references)),1)
print('보정된 유니그램 카운트 :',result)

보정된 유니그램 카운트 : {('the',): 1}


In [81]:
def modified_precision(candidate, reference_list, n):
    clip_cnt = count_clip(candidate, reference_list, n) 
    total_clip_cnt = sum(clip_cnt.values()) # 분자

    cnt = simple_count(candidate, n)
    total_cnt = sum(cnt.values()) # 분모

    # 분모가 0이 되는 것을 방지
    if total_cnt == 0: 
        total_cnt = 1

    # 분자 : count_clip의 합, 분모 : 단순 count의 합 ==> 보정된 정밀도
    return (total_clip_cnt / total_cnt)

In [82]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), n=1)
print('보정된 유니그램 정밀도 :',result)

보정된 유니그램 정밀도 : 0.14285714285714285


In [83]:
# 짧은 문장 길이에 대한 패널티(Brevity Penalty)

In [84]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    return closest_ref_len

In [85]:
def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1

    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
    elif ca_len == 0 :
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

In [86]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    # p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

In [87]:
# NLTK를 사용한 BLEU 측정하기

In [88]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('실습 코드의 BLEU :',bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

실습 코드의 BLEU : 0.2797821230630072
패키지 NLTK의 BLEU : 0.5045666840058485
